In [2]:
import pandas as pd
import json

In [ ]:
with open('reddit_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
df = pd.DataFrame(data)

df['created'] = pd.to_datetime(df['created'], unit='s')

df.set_index('created', inplace=True) # Make date index

df = df[(df.index.year == 2024) | (df.index.year == 2025)]
df = df[df['selftext'] != '']

df_posts = df[['id', 'selftext']].copy()

# Step 2: Add an empty column for manual labels
df_posts['manual_label'] = ''

# Step 3: Save the DataFrame to a CSV file
df_posts.to_csv('posts_to_label.csv', index=False)